# Watson OpenScale Fairness Metrics and Transformers - Performane Measures


The notebook will introduce the meaning of Fairness Metrics **Performance Measures** and show to compute Fairness Metrics **Performance Measures** based on the model prediction explain the meaning of then show how **Performance Measures**  can be used to transform the model output for fair prediction.<br/>

This document includes below sections:

- [1.Introduction of Perfromance Measures](#measures)
- [2.Setup Envrionments](#setup) 
- [3.Prepare input data](#input)
- [4.Configurations](#configuration)
- [5.Compute Perfromance Measures](#compute)

**Note:** This notebook should be run using with **Python 3.9.x** runtime. It requires service credentials for the following services:
  * Watson OpenScale <br/>

## 1. Introduction of Perfromance Measures <a name="measures"></a>

### Performance measures:

**Confusion matrix** is used to measure the performance of the classification model. It has a table of 4 different combinations. 

|  | Confusion Matrix |  |
| :-: | :-: | :-: |
| Actual\Predicted | Negative | Positive |
| Negative | TN | FP |
| Positive | FN | TP |

There are two things to noticed in the above image: 

    Predicted values: Values that are predicted by the model. 
    Actual Value: Values that are actually in a datasets.
    
Taking binary classification for understanding the model. Positive points belong to a positive class and Negative points to negative class. So it can be understood by these 4 points.

    True Positive(TP): Values that are actually positive and predicted positive.
    False Positive(FP): Values that are actually negative but predicted to positive.
    False Negative(FN): Values that are actually positive but predicted to negative.
    True Negative (TN): Values that are actually negative and predicted to negative.

Rate is a measure factor in a confusion matrix. It has also 4 basic types:

    True Positive Rate(TPR): True Positive/All Positive 

![title](https://latex.codecogs.com/svg.image?TPR&space;=&space;\frac{TP}{P}) 

    False Positive Rate(FPR): False Positive/All Negative 

![title](https://latex.codecogs.com/svg.image?FPR&space;=&space;\frac{FP}{N}) 

    False Negative Rate(FNR): False Negative/All Positive 

![title](https://latex.codecogs.com/svg.image?FNR&space;=&space;\frac{FN}{P})

    True Negative Rate(TNR): True Negative/All Negative 

![title](https://latex.codecogs.com/svg.image?TNR&space;=&space;\frac{TN}{N}) 

and 3 variant types:

    False Discovery Rate(FDR): False Positive/(True Positive+False Positive)
![title](https://latex.codecogs.com/svg.image?FDR&space;=&space;\frac{FP}{TP&plus;FP})

    False Omission Rate(FOR): False Negative/(True Negative+False Negative)
![title](https://latex.codecogs.com/svg.image?FOR&space;=&space;\frac{FN}{TN&plus;FN})

    Error Rate(ER): (False Positive+False Negative)/(All Positive + All Negative)
![title](https://latex.codecogs.com/svg.image?ER&space;=&space;\frac{FP&plus;FN}{P&plus;N}) 
 
Base on Confusion matrix, there are 12 metrics to measure the model performance:

**false_positive_rate_difference**: Returns the difference in FPR for the unprivileged and privileged groups. A value of 0 indicates equality of odds.

![title](https://latex.codecogs.com/svg.image?FPR_{D=unprivileged}-FPR_{D=privileged})

**false_positive_rate_ratio**: Returns the ratio of FPR for the unprivileged and privileged groups. A value of 1 indicates equality of odds.

![title](https://latex.codecogs.com/svg.image?\frac{FPR_{D=unprivileged}}{FPR_{D=privileged}})

**false_negative_rate_difference**: Returns the difference in FNR for the unprivileged and privileged groups. A value of 0 indicates equality of odds.

![title](https://latex.codecogs.com/svg.image?FDR_{D=unprivileged}-FDR_{D=privileged})

**false_negative_rate_ratio**: Returns the ratio of FNR for the unprivileged and privileged groups. A value of 1 indicates equality of odds.

![title](https://latex.codecogs.com/svg.image?\frac{FNR_{D=unprivileged}}{FNR_{D=privileged}})

**false_discovery_rate_difference**: Returns the difference in FDR for the unprivileged and privileged groups. A value of 0 indicates equality of odds.

![title](https://latex.codecogs.com/svg.image?FDR_{D=unprivileged}-FDR_{D=privileged})

**false_discovery_rate_ratio**: Returns the ratio of FDR for the unprivileged and privileged groups. A value of 1 indicates equality of odds.

![title](https://latex.codecogs.com/svg.image?\frac{FDR_{D=unprivileged}}{FDR_{D=privileged}})

**false_omission_rate_difference**: Returns the difference in FOR for the unprivileged and privileged groups. A value of 0 indicates equality of odds.

![title](https://latex.codecogs.com/svg.image?FOR_{D=unprivileged}-FOR_{D=privileged})

**false_omission_rate_ratio**: Returns the ratio of FOR for the unprivileged and privileged groups. A value of 1 indicates equality of odds.

![title](https://latex.codecogs.com/svg.image?\frac{FOR_{D=unprivileged}}{FOR_{D=privileged}})

**error_rate_difference**: Returns the difference in ER for the unprivileged and privileged groups. A value of 0 indicates equality of odds.

![title](https://latex.codecogs.com/svg.image?ER_{D=unprivileged}-ER_{D=privileged})

**error_rate_ratio**: Returns the ratio of ER for the unprivileged and privileged groups. A value of 1 indicates equality of odds.

![title](https://latex.codecogs.com/svg.image?\frac{ER_{D=unprivileged}}{ER_{D=privileged}}) 

**average_odds_difference**: Returns the average of the difference in FPR and TPR for the unprivileged and privileged groups. A value of 0 indicates equality of odds.

![title](https://latex.codecogs.com/svg.image?\frac{FPR_{D=unprivileged}-FPR_{D=privileged}&plus;TPR_{D=unprivileged}-TPR_{D=privileged}}{2})

**average_abs_odds_difference**: Returns the average of the absolute difference in FPR and TPR for the unprivileged and privileged groups. A value of 0 indicates equality of odds.

![title](https://latex.codecogs.com/svg.image?\frac{|FPR_{D=unprivileged}-FPR_{D=privileged}|&plus;|TPR_{D=unprivileged}-TPR_{D=privileged}|}{2})



  

## 2. Setup Envrionments <a name="setup"></a>

### 2.1 Package installation

*[Optional]* ignore warning messages to make output more clear.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install --upgrade ibm-watson-openscale --no-cache | tail -n 1
!pip install --upgrade ibm_metrics_plugin --no-cache | tail -n 1

#### Action: restart the kernel!

### 2.2 Configure credentials for WASTON OpenScale 
Configure credentials for WASTON OpenScale into the authenticator, which will be used in OpenScale client

In [ ]:
from ibm_cloud_sdk_core.authenticators import CloudPakForDataAuthenticator

WOS_CREDENTIALS = {
    "url": "<cluster-url>",
    "username": "<username>",
    "password": "<password>",
    "instance_id": "<openscale instance id>"
}

authenticator = CloudPakForDataAuthenticator(
    url=WOS_CREDENTIALS["url"],
    username=WOS_CREDENTIALS["username"],
    password=WOS_CREDENTIALS["password"],
    disable_ssl_verification=True
)

### 2.3 Setup OpenScale client 
Setup a Python OpenScale client with above setting.

In [ ]:
from ibm_watson_openscale import APIClient as OpenScaleAPIClient

client = OpenScaleAPIClient(
    service_url=WOS_CREDENTIALS['url'],
    service_instance_id=WOS_CREDENTIALS["instance_id"],
    authenticator=authenticator
)

client.version

## 3. Prepare input data <a name="input"></a>

### 3.1 Download and load the training data from github

In [ ]:
!rm german_credit_data_biased_training.csv
!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/Cloud%20Pak%20for%20Data/WML/assets/data/credit_risk/german_credit_data_biased_training.csv

In [ ]:
import pandas as pd

pandas_df = pd.read_csv("german_credit_data_biased_training.csv", sep=",", header=0)
pandas_df

### 3.2 Fake model predict result

In [ ]:
def _fake_predict_for_german_credit_dataset(row):
    if row['Telephone'] == 'yes':
        return 'No Risk'
    else:
        return 'Risk'
    
pandas_df["Predict"] = pandas_df.apply(lambda row: _fake_predict_for_german_credit_dataset(row), axis=1)
pandas_df

### 3.3 Create Spark Dataframe

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.config("spark.driver.bindAddress", "127.0.0.1").getOrCreate()
spark_df = spark.createDataFrame(pandas_df)

## 4. Configurations <a name="configuration"></a>

### Configuration Parameters

To setup configurations to compute Performance Measures, the Top level parameters:

- problem_type(enum): problem type. Possible values are "binary", "multiclass", "regression".
- label_column: the column where OpenScale to get the label value.
- fairness(dict(dict)): parameters for computing fairness metrics.

Parameters in fairness body:
- metrics_configuration(dict(dict)): metrics configuration. Format is a dict where the keys are FairnessMetricType and the values is a dict that includes metircs to calculated.
- protected_attributes: column list that will be kept and protected when processing data. 
- favourable_label: the positive outcomes in OpenScale, which could be find in label_column and predict_column.
- unfavourable_label: the negative outcomes in OpenScale, which could be find in label_column and predict_column.

Metric parameters in metrics_configuration:
- features(list): the column based on which to find out the unprivileged/privileged groups. it must be one at a time, or calculation will fail.
- predict_column: the column where OpenScale get the model prediction output.


Attributes prameters in protected_attributes:
- feature: feature column that need to be protect.
- reference_group: privileged groups at the systematic advantage, which could be find in feature column.
- monitored_group: unprivileged groups at the systematic advantage, which could be find in feature column.

In [ ]:
from ibm_metrics_plugin.common.utils.constants import FairnessMetricType

measure_configuration = {"configuration": {
            "problem_type": "binary",
            "label_column": "Risk",
            "fairness": {
                "metrics_configuration": {
                    FairnessMetricType.MEASURES.value: {
                        "average_odds_difference": {
                            "features": [["Sex"], ["Age"]],
                            "predict_column": "Predict"
                        },
                        "average_abs_odds_difference": {
                            "features": [["Sex"], ["Age"]],
                            "predict_column": "Predict"
                        },
                        "false_negative_rate_difference": {
                            "features": [["Sex"], ["Age"]],
                            "predict_column": "Predict"
                        },
                        "false_negative_rate_ratio": {
                            "features": [["Sex"], ["Age"]],
                            "predict_column": "Predict"
                        },
                        "false_positive_rate_difference": {
                            "features": [["Age"]],  
                            "predict_column": "Predict"
                        },
                        "false_positive_rate_ratio": {
                            "features": [["Sex"], ["Age"]],
                            "predict_column": "Predict"
                        },
                        "false_discovery_rate_ratio": {
                            "features": [["Sex"], ["Age"]],
                            "predict_column": "Predict"
                        },
                        "false_discovery_rate_difference": {
                            "features": [["Sex"]],  
                            "predict_column": "Predict"
                        },
                        "false_omission_rate_difference": {
                            "features": [["Sex"], ["Age"]],
                            "predict_column": "Predict"
                        },
                        "false_omission_rate_ratio": {
                            "features": [["Sex"], ["Age"]],
                            "predict_column": "Predict"
                        },
                        "error_rate_difference": {
                            "features": [["Sex"], ["Age"]],
                            "predict_column": "Predict"
                        },
                        "error_rate_ratio": {
                            "features": [["Sex"], ["Age"]],
                            "predict_column": "Predict"
                        }
                    }
                },
                "protected_attributes": [
                    {
                        "feature": "Sex",
                        "reference_group": ["male"],
                        "monitored_group": ["female"]
                    },
                    {
                        "feature": "Age",
                        "reference_group": [[26, 55], [56, 75]],
                        "monitored_group": [[55, 60]]
                    }
                ],
                "favourable_label": ["No Risk"],
                "unfavourable_label": ["Risk"],
            }
        }}

## 5. Compute Perfromance Measures <a name="compute"></a>

calculate performance measures

In [ ]:
metrics = client.ai_metrics.compute_metrics(spark=spark, configuration=measure_configuration, data_frame=spark_df)

show the result 

In [ ]:
import pprint
pprint.pprint(metrics.get("metrics_result").get("fairness").get("performance_measures"))